In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")

In [12]:
df = pd.read_csv("train.csv");

In [13]:
print df.shape

(250306, 34)


In [6]:
import gmplot

Remove data points where violator was found not responsible.

In [14]:
df = df[~ df.compliance.isnull()]
print df.shape

(159880, 34)


In [15]:
df.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0


In [16]:
addresses = pd.read_csv("addresses.csv")
addresses.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [20]:
address_dict = {}
for index, row in addresses.iterrows():
    address_dict[row["ticket_id"]] = row["address"]

In [24]:
df["address"] = df.ticket_id.apply(lambda x: address_dict[x])

In [26]:
latlons = pd.read_csv("latlons.csv")
print latlons.shape

(121769, 3)


In [41]:
detroit = pd.merge(df, latlons, on="address", how="inner")
print detroit.shape

(159880, 37)


In [42]:
exclude = ["payment_amount", "payment_date", "payment_status", "balance_due", "collection_status", "compliance_detail"]

In [43]:
detroit = detroit.drop(exclude, axis=1)
print detroit.shape

(159880, 31)


In [50]:
detMap = gmplot.GoogleMapPlotter(42.3314, -83.0458, 16)
detMap.heatmap(detroit.lat, detroit.lon)
detMap.draw("DetroitBlight.html")

In [57]:
from sklearn.linear_model import LogisticRegression

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve

In [118]:
detroitClean = detroit[~ detroit.mailing_address_str_number.isnull() & ~ detroit.lat.isnull() & ~ detroit.lon.isnull()]
X = detroitClean.drop(["compliance", "address", "agency_name", "violator_name", "violation_description", "inspector_name", 
                 "mailing_address_str_name", "disposition", "ticket_issued_date", "hearing_date", "state", "city", 
                 "non_us_str_code", "country", "violation_street_name", "violation_code", "zip_code", 
                "violation_zip_code", "grafitti_status"], axis=1)
y = detroitClean["compliance"]

In [119]:
print X.shape, y.shape

(157320, 12) (157320L,)


In [102]:
#X.zip_code = X.zip_code.apply(lambda x: int(x))

In [120]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y)
print xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

(117990, 12) (39330, 12) (117990L,) (39330L,)


In [134]:
lreg = LogisticRegression(C=25)
lreg.fit(xtrain, ytrain)

LogisticRegression(C=25, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [135]:
preds = lreg.predict_proba(xtest)[:, 0]
fpr, tpr, thresholds = roc_curve(ytest, preds, pos_label=1)
print "Area under curve:", auc(fpr, tpr)

Area under curve: 0.238916099322


In [137]:
from sklearn.ensemble import RandomForestClassifier

In [140]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(xtrain, ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [141]:
preds = rf.predict_proba(xtest)[:, 0]
fpr, tpr, thresholds = roc_curve(ytest, preds, pos_label=1)
print "Area under curve:", auc(fpr, tpr)

Area under curve: 0.169979257477
